<a href="https://colab.research.google.com/github/camdsDS/Cesar_Portfolio/blob/main/HousePrices_from_Kaggle_RandomForestRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Dados do kaggle: House Prices - Advanced Regression Techniques
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [ ]:
data_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
#Features
data_train.columns
features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition']

In [ ]:
#definindo o alvo, as features, o treino e o teste
y = data_train['SalePrice']
train = data_train[features]
test = data_test[features]

In [ ]:
#a coluna MSSubClass é categórica, porém os dados são números.
#Para ficar claro, vou inserir um 'A' na frente de cada dado
train['MSSubClass'] = [str(x)+'A' for x in train['MSSubClass']]
test['MSSubClass'] = [str(x)+'A' for x in test['MSSubClass']]
#poderia ter feito também por
#train['MSSubClass'] = train['MSSubClass'].apply(lambda x: str(x) +'A')

In [ ]:
#função que dropa colunas com uma porcentagem de valores miss superior a um valor P_miss arbitrário
#P_miss é a menor % tolerada de miss
def DropaMiss(df, P_miss=0):
  df_miss2 = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index().sort_values('P_miss', ascending = False)
  df_miss = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index()
  df_miss = df_miss[df_miss['P_miss'] <= P_miss]
  df = df[df_miss['index'].to_list()]
  return df,df_miss2

In [ ]:
train, relatorio_miss = DropaMiss(train, 80)

In [ ]:
relatorio_miss
#Daqui vemos que as colunas dropadas foram ['PoolQC', 'MiscFeature', 'Alley', 'Fence']

,index,P_miss
71,PoolQC,99.520548
73,MiscFeature,96.301370
5,Alley,93.767123
72,Fence,80.753425
56,FireplaceQu,47.260274
...,...,...
38,Heating,0.000000
1,MSZoning,0.000000
40,CentralAir,0.000000
42,1stFlrSF,0.000000


In [ ]:
#vamos dropar no df test as mesmas colunas que dropamos no df train
test = test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)

In [ ]:
#dividindo o df train em dois df: numerico e categorico
train.dtypes
#Temos apenas 3 tipos de dados: int64, float64 e object
numerico_train = train.select_dtypes(exclude=['object'])
categorico_train = train.select_dtypes('object')

In [ ]:
#dividindo o df test em dois df: numerico e categorico de modo que tenha
#as mesmas colunas dos novos df de train
numerico_test = test[numerico_train.columns]
categorico_test = test[categorico_train.columns]

In [ ]:
#tratando missings do df numerico do treino
numerico_train.isna().sum()

LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [ ]:
valores_train = {'LotFrontage':numerico_train['LotFrontage'].mean(), 'MasVnrArea':numerico_train['MasVnrArea'].mean(), 'GarageYrBlt':numerico_train['GarageYrBlt'].mean()}
numerico_train = numerico_train.fillna(value=valores_train)

In [ ]:
#vamos tratar os missings do df numerica do test
numerico_test.isna().sum()

LotFrontage      227
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        15
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       78
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [ ]:
valores_test = {'LotFrontage':numerico_train['LotFrontage'].mean(), 'MasVnrArea':numerico_train['MasVnrArea'].mean(), 'BsmtFinSF1':numerico_train['BsmtFinSF1'].mean(), 'BsmtFinSF2':numerico_train['BsmtFinSF2'].mean(),
                'BsmtUnfSF':numerico_train['BsmtUnfSF'].mean(), 'TotalBsmtSF':numerico_train['TotalBsmtSF'].mean(), 'BsmtFullBath':numerico_train['BsmtFullBath'].mean(), 'BsmtHalfBath':numerico_train['BsmtHalfBath'].mean(),
                'GarageYrBlt':numerico_train['GarageYrBlt'].mean(), 'GarageCars':numerico_train['GarageCars'].mean(), 'GarageArea':numerico_train['GarageArea'].mean()}
numerico_test = numerico_test.fillna(value=valores_test)

In [ ]:
#get_dummies na base base dos categoricos
categorico_train = pd.get_dummies(categorico_train, drop_first=False, dtype=None)
categorico_test = pd.get_dummies(categorico_test, drop_first=False, dtype=None)

In [ ]:
#criando listas para comparar os df train e test
lista_colunas_train = categorico_train.columns.to_list()
lista_colunas_test = categorico_test.columns.to_list()

In [ ]:
#comparando (vendo o que está no train e não está no test)
Z = set(lista_colunas_train) - set(lista_colunas_test)
Z

{'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'Electrical_Mix',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_Other',
 'GarageQual_Ex',
 'Heating_Floor',
 'Heating_OthW',
 'HouseStyle_2.5Fin',
 'RoofMatl_ClyTile',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'Utilities_NoSeWa'}

In [ ]:
#criando colunas no df test que tinham no train, mas não no test 
for col in Z:
  categorico_test[col] = 0

In [ ]:
#atualizando
lista_colunas_train = categorico_train.columns.to_list()
lista_colunas_test = categorico_test.columns.to_list()

In [ ]:
#comparando (vendo o que está no test e não está no train)
Z = set(lista_colunas_test) - set(lista_colunas_train)
Z

{'MSSubClass_150A'}

In [ ]:
#dropando do test o que está no test e não está no train
categorico_test = categorico_test.drop('MSSubClass_150A', axis = 1)

In [ ]:
#atualizando
lista_colunas_train = categorico_train.columns.to_list()
lista_colunas_test = categorico_test.columns.to_list()

In [ ]:
#verificando se agora as colunas estão iguais
set(lista_colunas_test) == set(lista_colunas_train)

True

In [ ]:
#Fazendo o merge dos dataframes
abt_Train = pd.merge(numerico_train, categorico_train, how ='left', on=None, left_on=None, right_on=None, left_index=True, right_index=True)
abt_Test = pd.merge(numerico_test, categorico_test, how ='left', on=None, left_on=None, right_on=None, left_index=True, right_index=True)

In [ ]:
#normalização do df train
scaler = MinMaxScaler()
scaler.fit(abt_Train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
abt_train = pd.DataFrame(scaler.transform(abt_Train), columns=abt_Train.columns)

In [ ]:
#normalização do df test
scaler2 = MinMaxScaler()
scaler2.fit(abt_Test)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
abt_test = pd.DataFrame(scaler2.transform(abt_Test), columns=abt_Test.columns)

In [ ]:
#Criando e rodando o modelo

In [ ]:
desenvolvimento = abt_train.copy()
X_test = abt_test.copy()

In [ ]:
#dividindo nosso train em desenvolvimento e validação
X_desenv, X_valid, y_desenv, y_valid = train_test_split(desenvolvimento, y, test_size = 0.7)

In [ ]:
#vamos usar o algoritmo RandomForestRegressor() e vamos usar
#o GridSearchCV para buscar bons valores para 'max_depth' e 'n_estimators'
busca_modelo = GridSearchCV(estimator = RandomForestRegressor(), param_grid={'max_depth' : [8, 16, 24], 'n_estimators' : [500, 1000, 1500]})

In [ ]:
busca_modelo.fit(X_desenv, y_desenv)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

In [ ]:
model = busca_modelo.best_estimator_

In [ ]:
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=24, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
model.fit(X_desenv, y_desenv)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=24, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
erro = mean_squared_error(y_valid, model.predict(X_valid))
np.sqrt(erro)

30504.737785845842

In [ ]:
previsao = model.predict(X_test)

In [ ]:
#gerando csv resposta
final = pd.read_csv('test.csv')
final['SalePrice'] = previsao
final = final[['Id', 'SalePrice']]
final.to_csv('DesafioHousePriceF.csv', index = False, sep = ',')